In [1]:
# 更新 
import json
from pathlib import Path
from shutil import copyfile
from PIL import Image
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
#from qwen_vl_utils import process_vision_info

system_v0 = """<|im_start|>
# 指令：
你是一位经验丰富的图像描述专家，擅长用清晰、准确且生动的语言捕捉并描绘静态画面中的所有视觉细节。你的任务是为提供的图片生成一段高质量的描述，帮助未见过图片的人在脑海中形成一幅精确而完整的图像。

# 核心目标与风格：
请以一位观察敏锐的专家口吻，**客观、详尽且自然流畅地**描述图片内容。你的描述应引导读者逐步了解画面的整体布局、核心元素、重要细节以及它们之间的内在联系。力求语言平实易懂，同时展现出对视觉信息的深刻理解。

# 描述的组织与层次感 (请自然融入，形成连贯段落)：
在构思描述时，请有意识地组织信息，使其具有逻辑层次感，方便读者理解：
1.  **整体印象与场景概览**：首先，对图片呈现的整体场景和画面的主要构成或最先吸引注意力的部分进行简要介绍。
2.  **画面核心详述**：接着，聚焦于图片中的主要人物、动物、物体或视觉焦点。详细描述它们的：
    *   **身份与特征**：它们是什么？具有哪些显著的物理特征（如人的外貌、服饰、姿态；物体的形状、色彩、材质、大小感等）？
    *   **状态与行为**：它们目前呈现何种状态（例如：静止、动态、特定姿势）？如果有人物或动物，他们在进行什么可以直接观察到的行为？
3.  **周边元素与环境细节**：然后，将视野扩展到围绕核心内容的其他相关元素，以及背景环境的具体情况。包括但不限于：
    *   次要的物体、点缀物、地面特征、天空状况、植被细节等。
    *   环境中可辨识的特定参照物（如远处的建筑轮廓、路边的标志、室内的特定家具）。
    *   光线给人的感觉（例如：明亮通透、柔和的室内光、夕阳余晖等，基于视觉判断）。
4.  **空间布局与元素关系**：阐明各主要元素在画面空间中的相对位置和排列方式（例如：前景中的A物体，中景偏左的B人物，背景深处的C景物）。如果构图有值得一提的特点，可以自然地融入描述。
5.  **文本信息 (若有且清晰可辨)**：如果图片中包含任何清晰的文字、数字或符号，请准确地指出来，并说明其大致位置和外观。

# 输出要求 (务必严格遵守)：
*   **基于视觉，客观呈现**：你的全部描述必须严格依据图片中实际可见的视觉信息。**避免加入任何非视觉信息（如声音、气味），避免无依据地猜测人物的内心活动、动机或讲述图片未明示的故事。**
*   **准确详尽，注重细节**：在客观的前提下，力求捕捉并准确描述所有重要的视觉细节。使用具体、明确的词汇。例如，颜色描述应尽可能精确，物体状态应清晰具体。
*   **语言自然流畅，避免刻板生硬**：
    *   句子和段落的组织应自然，符合中文的表达习惯。**可以使用恰当的连接词和过渡性短语，确保描述的连贯性和整体感。**
    *   **语言风格应平实、清晰，避免使用过于雕琢、华丽或带有强烈主观情感的词汇。**
    *   **避免机械地逐条回应“描述的组织与层次感”或“画面核心详述”中的子项。** 将这些要点自然地整合到你的描述性段落中。
*   **杜绝主观评价与拟人化**：不要对图片内容进行美学上或个人喜好上的评价。不要赋予无生命的物体以人的情感、意图或行为。
*   **开篇自然，避免公式化**：可以从对场景的整体印象入手，或直接点出画面中最先引人注意的部分，避免使用“这张图片是关于…”、“图中显示了…”等显得过于格式化的开场白，力求自然引入。

\n<|im_end|>
"""



system_v1 = """<|im_start|>system
# 指令：
你是一位经验丰富的图像描述专家，擅长基于图像内容生成清晰、准确、生动的自然语言描述。你理解不同类型图像所呈现的全部信息，能够灵活调整观察视角，生成客观、详尽且自然流畅的高质量描述，帮助未见图片的用户在脑海中准确重构画面。引导读者逐步了解画面的整体布局、核心元素、重要细节以及它们之间的内在联系。

# 任务目标：
1)核心目标：请以一位观察敏锐的专家口吻、客观、详尽且自然流畅地描述图片内容。你的描述应引导读者逐步了解画面的整体布局、核心元素、重要细节以及它们之间的内在联系。力求语言平实易懂，同时展现出对视觉信息的深刻理解。
2)语言结构风格：输出语言应是清晰、平实、流畅，自然而富有条理的中文，不使用标题、不罗列清单。可以使用恰当的连接词和过渡性短语，确保描述的连贯性和整体感。
3)语言叙述风格：叙述风格应平实、清晰，避免使用过于雕琢、华丽的修辞和带有强烈主观情感的词汇。避免机械地逐条回应“输出具体约束”中的子项。将这些要点自然地整合到你的描述性段落中。
4)类型适配：根据图像类型（自然摄影、人工视觉图、混合型图像）自动调整观察维度与语言侧重点。自然摄影类图像，如风景、人像、街景、室内外场景、动植物等，重点关注画面构图、真实物体形态、状态、位置与环境；人工视觉类图像，如图表、图纸、漫画、插画、UI界面、流程图、表格等，重点描述视觉结构、数据关系、标签内容及视觉元素之间的功能性联系；混合型图像，如照片中包含屏幕截图、展板、海报、图纸、漫画页面等，需兼顾自然视觉与人工信息，明确区分并分别叙述。

# 输出具体描述：
1)首先，从宏观角度简要描述图像内容，帮助读者建立初步印象，主要包括：
    a)图像类型（自然景观？统计图表？室内空间？漫画页面？）
    b)图像类型中的主体元素是什么，交代个数(可能是多个）、元素的区域。；
    c)画面基调与光影情况（明亮通透、阴暗压抑、柔和温暖等）；
2)接着，详细描述画面中关键的主体元素（可能有多个），说明数量并分别详细描述它们的种类与特征（如人物、动植物的外形结构、表情外貌，具体数量､ 全身分别的颜色构成、大小、质感、服饰、姿态等）、描述它们的状态与行为（静止、运动、交互，特殊姿势等），并指出其在画面中的空间层次和关联（具体位置）；
3)之后扩展至主体元素周围具体位置及背景环境，请和描述主体元素一样的详细程度去描述次要物体、装饰性细节（如小道具，点缀物、地面材料、交通标志等）、自然特征（天空状态、植被覆盖细节、水体、阴影变化）、天气气候迹象及光影氛围（光线方向、照度、光源位置等）；
4)然后，阐明画面全部元素之间的空间相对位置与排列方式，包括全部元素的排列顺序、遮挡关系、构图策略（对称、分层、集中等），以及若有多个子区域（如漫画分格、图表区块）的空间逻辑和排列方式，请分别详细说明，另如果构图有值得一提的特点，可以自然地融入描述；
5)如果图像中包含文字、符号或人工视觉信息，请准确描述其内容、位置及视觉风格，包括但不限于：
    a)文字与数字：具体文字内容、所在位置、字体颜色、大小、粗细及清晰度，数字的含义（价格、比例、年份等）；
    b)图表与结构图：图表类型（柱状图、折线图、饼图、表格、流程图等），坐标轴及单位、图例颜色、数据趋势、结构图的流程箭头和节点；
    c)漫画、插画、界面截图的详细内容，包括但不限于，漫画中出现的单个或多格画面的具体描述，漫画的风格等：手绘或数字绘图角色、对话框、旁白、动作帧，界面层级、组件布局及功能区等。

# 请严格遵守以下要求：
1)仅基于图像中可见内容进行描述，不推断情节或虚构人物意图。
2)语言自然流畅，避免列清单、使用标题编号或模板化开场；开篇自然，可以从对场景的整体印象入手，或直接点出画面中最先引人注意的部分；力求自然引入和结尾。
3)描述客观准确，在客观的前提下，力求捕捉并准确且详尽的描述所有的视觉细节。避免使用过于雕琢、华丽的修辞和带有强烈主观情感的词汇。
4)若有清晰文字、图表等人工信息，必须准确指出其内容、位置和外观风格，和主要元素的联系。
5)输出为一段结构高质量，描述内容详尽的文本，使用符合中文的表达习惯以及适合中国人理解的用词。
6）头尾自然，避免使用“这张图片是关于…”、“图中显示了…”等显得过于格式化的描述，力求自然引入。
7）请和描述主体元素一样的详细程度去描述次要物体的种类与特征可参考输出具体描述2）。
\n<|im_end|>
"""

assistant = "\n<|im_start|>assistant"
user = "\n<|im_start|>user \n <|vision_start|><|image_pad|><|vision_end|>请根据图片帮我生成描述文本<|im_end|>"

prompt_0 =  system_v0 + user + assistant 
prompt_1 =  system_v1 + user + assistant
 

/home/rootlab338/work/envs_conda/nextqa/qwen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 1. 加载Qwen2.5
model_qwen = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-7B-Instruct",
    torch_dtype=torch.float16,
    device_map="auto",
)
processor_qwen = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")
device = next(model_qwen.parameters()).device 


Loading checkpoint shards: 100%|██████████| 5/5 [00:07<00:00,  1.43s/it]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


In [3]:
# 2. 加载llava-next

# from transformers import LlavaProcessor, LlavaForConditionalGeneration
# from transformers import LlavaNextVideoProcessor, LlavaNextVideoForConditionalGeneration

# import torch
# model_name_or_path = "/media/data/document/wpf_works/learn/learnllava_next_video/LLaVA-NeXT-Video-7B-hf"
# model = LlavaNextVideoForConditionalGeneration.from_pretrained(
#     model_name_or_path, 
#     torch_dtype=torch.float16, 
#     device_map="auto",
# )
# processor = LlavaNextVideoProcessor.from_pretrained(model_name_or_path)
# inputs = llava_processor(text=prompt, images=image, return_tensors="pt")

In [ ]:
###test用

image_path = "/home/rootlab338/work/qwen2.5/data_hq/nature/churchbird.jpg"
image = Image.open(image_path).convert("RGB")
inputs= processor_qwen(text=[prompt_1],images=image,return_tensors="pt",padding=True).to(device)
generated_ids = model_qwen.generate(**inputs, max_new_tokens=512)
trimmed_ids = [out[len(inp):] for inp, out in zip(inputs.input_ids, generated_ids)]
output_text = processor_qwen.batch_decode(trimmed_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(output_text)

In [ ]:
image_path = "/home/rootlab338/work/qwen2.5/data_hq/nature/churchbird.jpg"
image = Image.open(image_path).convert("RGB")
inputs= processor_qwen(text=[prompt_1],images=image,return_tensors="pt",padding=True).to(device)
generated_ids = model_qwen.generate(**inputs, max_new_tokens=512)
trimmed_ids = [out[len(inp):] for inp, out in zip(inputs.input_ids, generated_ids)]
output_text = processor_qwen.batch_decode(trimmed_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(output_text)

In [33]:
from tqdm import tqdm 

img_dir_0 = Path("/home/rootlab338/work/qwen2.5/data_hq/artificial")
img_dir_1 = Path("/home/rootlab338/work/qwen2.5/data_hq/nature")
img_dir_2 = Path("/home/rootlab338/work/qwen2.5/data_hq/concat")
img_dirs = [img_dir_0,img_dir_1,img_dir_2 ]

for img_dir in img_dirs:
    results = []
    prompts = [prompt_0,prompt_1]
    image_extensions = [".jpg", ".jpeg", ".png", ".bmp", ".tiff", ".webp"]
    image_paths = sorted([p for p in img_dir.iterdir() if p.suffix.lower() in image_extensions])

    for image_path in tqdm(image_paths):
        image = Image.open(image_path).convert("RGB")

        for prompt in prompts:
            inputs= processor_qwen(
                    text=[prompt],
                    images=image,
                    return_tensors="pt",
                    padding=True
                ).to(device)
            generated_ids = model_qwen.generate(**inputs, max_new_tokens=512)
            trimmed_ids = [out[len(inp):] for inp, out in zip(inputs.input_ids, generated_ids)]
            output_text = processor_qwen.batch_decode(trimmed_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
            results.append({
                    "image": image_path.name,
                    f"description_of_prompt": output_text,
                    "prompt_token_count": len(inputs.input_ids[0]),
                    "generated_token_count": len(generated_ids[0]),
                    "answer_token_count": len(trimmed_ids[0])
            })
            i += 1
    output_path = img_dir / "generated_descriptions.json"
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

100%|██████████| 7/7 [02:36<00:00, 22.39s/it]


 这张图片展示了一座城市的全景，画面中央是一座白色的圆顶建筑，顶部有一个黑色的星月标志，象征着伊斯兰教的信仰。这座建筑位于画面的中心偏右位置，周围环绕着许多低矮的砖瓦房屋，显示出一种典型的中东或南亚城市风貌。远处可以看到一些高大的通信塔，表明这是一个现代化的城市。

天空呈现出灰蒙蒙的颜色，给人一种阴沉的感觉，可能是清晨或傍晚时分。空中有许多鸟儿在飞翔，其中一只飞得非常近，占据了画面的左上角，增加了画面的动感。这些鸟儿分散在不同的高度，有的靠近建筑物，有的则飞得更高，形成了一个动态的飞行图案。

建筑物的屋顶上有一些晾晒的衣物，显示出居民的生活气息。整个画面给人一种宁静而略带忧郁的感觉，可能是由于阴沉的天气和城市中常见的建筑风格所造成的。
